In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_dynamics.jl")
# include("./funlopt/funl_constraint.jl")
# include("./trajopt/scaling.jl")

diff_numeric (generic function with 1 method)

In [3]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/quadstar_N15_traj" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"];
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [6]:
@load "./data/quadstar_N15_funl" my_dict
Qnom = my_dict["Q"]
Knom = my_dict["K"]
Znom = my_dict["Z"];
Snom = my_dict["S"]

1×16 Matrix{Float64}:
 6.23775e-7  1.20409e-6  2.62165e-5  …  0.00465642  0.00466491  0.00467466

In [7]:
dynamics = QuadrotorDynamics()
ix = dynamics.ix
iu = dynamics.iu
decay_rate = 0.1
DLMI = NonlinearDLMI(decay_rate,ix,iu,dynamics.Cv,dynamics.Dvu)
is = DLMI.is

1

In [9]:
Xnom,Unom = QKZS_to_XU(Qnom,Knom,Znom,Snom)

([21.921719532381484 16.4586677163902 … 0.07788508645602667 0.03893804652670378; -0.502783174767715 -0.4136642564936496 … -0.002313509066533346 0.00026194868609703186; … ; 0.027011559399437907 -0.12186454284038221 … -0.003456518419072749 5.7704292519148744e-5; 1.4408069885406187 0.3492424991399964 … 0.0029519861403257143 0.0011616726449049557], [-0.042517273946081846 0.3679914079919505 … -0.5181838433608075 0.8168578224861411; 0.0017639863408449269 0.022891272346612285 … -0.010621376112180166 0.010034226245156794; … ; 3.659446564066534e-5 3.924857583081413e-6 … 3.652293085267136e-5 8.556040160733173e-5; 6.237746133772455e-7 1.204085094190933e-6 … 0.00466491353373636 0.004674655882089802])

In [11]:
print_jl(Xnom)
print_jl(Unom)

Type is Matrix{Float64}
Shape is (78, 16)
Type is Matrix{Float64}
Shape is (127, 16)


In [15]:
A,Bm,Bp,s,z,x_prop,X_prop = discretize_foh(DLMI,dynamics,xnom[:,1:N],unom,dtnom,Xnom[:,1:N],Unom)

([0.8611561637452997 0.03684914029621754 … -0.06491946070853909 0.0021932546243765166; -0.0052903033688221366 0.8656278902079806 … -0.02477842817527878 0.0020466480582557655; … ; 0.0002556191428782296 -0.0008306295761696502 … 0.0325381366963203 -0.009157968093256306; 7.367454046486404e-5 -0.00031637222181204485 … 0.009390400296463567 0.013982262559628589;;; 0.8003637077751435 0.035587954975204325 … 0.15962101474025173 0.10408953305308288; -0.003941630203395363 0.7090109495576278 … -0.13991297895485605 -0.017943903070313145; … ; 0.0002722202527683991 0.0005955781872750731 … -0.04269180360780222 -0.0032833707133386467; 0.0003280508426778697 0.001958711246042137 … 0.0025303000945962007 0.00010176348637264463;;; 0.9384367462858172 0.3150533374534279 … 0.10149295334146431 0.0034942537629076803; 0.10142151105741086 0.8210187571703262 … -0.03034389072586783 -0.000779272666772313; … ; 0.11847563260070103 -0.09976698012376363 … 0.07772823566896313 0.024523865625968706; 0.09345524337680843 0.026

In [16]:
print_jl(A)

Type is Array{Float64, 3}
Shape is (78, 78, 15)


In [17]:
function dvdt(out,V,p,t)
    um = p[1]
    up = p[2]
    Um = p[3]
    Up = p[4]
    dt = p[5]

    alpha = (dt - t) / dt
    beta = t / dt

    u_ = alpha * um + beta * up
    U_ = alpha * Um + beta * Up

    x_ = V[idx_x]
    X_ = V[idx_X]

    # traj terms
    f = forward(dynamics,x_,u_)
    fx,fu = diff(dynamics,x_,u_)

    # funl terms
    F = forward(model,X_,U_,fx,fu)

    dV = [f;F]
    out .= dV[:]
end
function forward_RK4(dvdt,X,U,xnom,unom,dtnom)
    V0 = [xfwd[:,i];xnom[:,i]][:]
    um = unom[:,i]
    up = unom[:,i+1]
    km = vec(K[:,:,i])
    kp = vec(K[:,:,i+1])
    dt = Tnom[i]

    prob = ODEProblem(dvdt,V0,(0,1),(um,up,km,kp,dt))
end

dvdt (generic function with 1 method)

In [ ]:
V0 = [xfwd[:,i];xnom[:,i]][:]
um = unom[:,i]
up = unom[:,i+1]
km = vec(K[:,:,i])
kp = vec(K[:,:,i+1])
dt = Tnom[i]

prob = ODEProblem(dvdt,V0,(0,dt),(um,up,km,kp,dt))

In [ ]:
function diff_numeric(model::FunnelDynamics,x::Vector,u::Vector,A::Matrix,B::Matrix)
    ix = length(x)
    iu = length(u)
    eps_x = Diagonal{Float64}(I, ix)
    eps_u = Diagonal{Float64}(I, iu)
    fx = zeros(ix,ix)
    fu = zeros(ix,iu)
    h = 2^(-18)
    for i in 1:ix
        fx[:,i] = (forward(model,x+h*eps_x[:,i],u,A,B) - forward(model,x-h*eps_x[:,i],u,A,B)) / (2*h)
    end
    for i in 1:iu
        fu[:,i] = (forward(model,x,u+h*eps_u[:,i],A,B) - forward(model,x,u-h*eps_u[:,i],A,B)) / (2*h)
    end
    return fx,fu
end